In [10]:
!pip install sentence-transformers transformers peft torch

In [11]:


from transformers import AutoModelForQuestionAnswering, AutoTokenizer
from peft import AdaLoraConfig, get_peft_model
from torch.utils.data import Dataset
from transformers import Trainer, TrainingArguments
import json
import torch

class TaskSpecificFineTuning:
    def __init__(self, model_path="google/flan-t5-small"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = AutoModelForQuestionAnswering.from_pretrained(model_path)
        self.configure_peft_adapter()

    def configure_peft_adapter(self, verbose=True) -> None:
        """Configure the PEFT adapter."""
        self.adapter_config = AdaLoraConfig(target_r=16)
        self.model = get_peft_model(self.model, self.adapter_config)
        if verbose:
            self.model.print_trainable_parameters()

    def ask_question(self, question, context, device="cpu"):
        """Tokenize the input and predict the answer."""
        inputs = self.tokenizer.encode_plus(
            question, context, add_special_tokens=True, return_tensors="pt"
        )

        # Adjustments for device placement
        device = torch.device(device)
        self.model.to(device)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Get model predictions
        with torch.no_grad():
            outputs = self.model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])

        # Get the start and end positions
        answer_start_scores = outputs.start_logits
        answer_end_scores = outputs.end_logits

        # Find the tokens with the highest `start` and `end` scores
        answer_start = torch.argmax(answer_start_scores)
        answer_end = torch.argmax(answer_end_scores) + 1

        # Convert the tokens to the answer string
        answer = self.tokenizer.convert_tokens_to_string(
            self.tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end])
        )
        return answer


class StylesprintDataset(Dataset):
    def __init__(self, tokenizer, data):
        tokenizer.pad_token = tokenizer.eos_token
        self.tokenizer = tokenizer
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        question, answer = self.data[idx]["question"], self.data[idx]["answer"]

        # Tokenize the pair
        encoding = self.tokenizer.encode_plus(
            question,
            answer,
            add_special_tokens=True,
            max_length=512,
            padding="max_length",
            truncation=True,
            return_offsets_mapping=True,
        )
        input_ids = encoding["input_ids"]
        attention_mask = encoding["attention_mask"]
        offset_mapping = encoding["offset_mapping"]

        # Initialize start and end positions to None
        start_positions = None
        end_positions = None

        # Find the start and end of the answer in the tokenized sequence
        for i, offset in enumerate(offset_mapping):
            if (
                start_positions is None
                and offset[0] == 0
                and self.tokenizer.decode([input_ids[i]]).strip() == answer.split()[0]
            ):
                start_positions = i
            if (
                offset[1] == len(answer)
                and self.tokenizer.decode([input_ids[i]]).strip() == answer.split()[-1]
            ):
                end_positions = i

        # Ensure that start and end positions are set
        if start_positions is None or end_positions is None:
            start_positions = 0
            end_positions = 0

        # Return the inputs and positions
        return {
            "input_ids": torch.tensor(input_ids),
            "attention_mask": torch.tensor(attention_mask),
            "start_positions": torch.tensor(start_positions),
            "end_positions": torch.tensor(end_positions),
        }


if __name__ == "__main__":
    from google.colab import drive
    drive.mount('/content/drive')

    ts = TaskSpecificFineTuning("google/flan-t5-base")

    # Create some demo data (replace this with your actual data)
    demo_data = [
        {"question": "Can I exchange an online purchase?", "answer": "Yes, within 30 days"},
        {"question": "What's the return policy?", "answer": "30 days from purchase"},
        # Add more question-answer pairs as needed
    ]

    # Split the mock dataset into training and evaluation sets (50/50)
    train_data = StylesprintDataset(ts.tokenizer, demo_data[: len(demo_data) // 2])
    eval_data = StylesprintDataset(ts.tokenizer, demo_data[len(demo_data) // 2 :])

    # Training arguments
    training_args = TrainingArguments(
        output_dir="/content/drive/MyDrive/stylesprint_qa_results",
        num_train_epochs=20,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=64,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir="/content/drive/MyDrive/stylesprint_qa_logs",
        logging_steps=10,
    )

    # Initialize the Trainer
    trainer = Trainer(
        model=ts.model,
        args=training_args,
        train_dataset=train_data,
        eval_dataset=eval_data,
    )

    # Start training
    trainer.train()

    # Save the model
    ts.model.save_pretrained("/content/drive/MyDrive/stylesprint_qa_model")
    ts.tokenizer.save_pretrained("/content/drive/MyDrive/stylesprint_qa_model")

    # Evaluate the model
    question = "Can I exchange an online purchase?"

    # Example context (You can use any relevant text)
    context = """
    At Stylesprint, we strive to ensure the utmost satisfaction for all our customers. Our return and exchange policy is crafted to provide you with a seamless and convenient shopping experience. If you're not completely satisfied with your purchase, you can return or exchange your items within 30 days from the date of purchase. To be eligible for a return or exchange, items must be in their original, unworn condition with all tags attached. Footwear returns must include the original shoebox in its original condition. We request that you provide a valid proof of purchase with any return. Refunds will be processed to the original method of payment and may take up to two billing cycles to appear on your credit card statement.
    In the case of exchanges, the availability of your desired item will be confirmed upon processing. If the item is not available, we will issue a refund instead. Please note that sale items are only eligible for exchange and not for refunds. Our aim is to make your shopping experience as enjoyable as possible, and our dedicated customer service team is always here to assist you with any concerns or questions you may have regarding our return policy.
    """

    answer = ts.ask_question(
        question, context, device="cuda" if torch.cuda.is_available() else "cpu"
    )
    print("Question:", question)
    print("Answer:", answer)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of T5ForQuestionAnswering were not initialized from the model checkpoint at google/flan-t5-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 3,467,232 || all params: 226,372,490 || trainable%: 1.5316


Step,Training Loss
10,6.321200
20,6.322800


Question: Can I exchange an online purchase?
Answer: a refund instead. Please note that sale items are only eligible for exchange and not for refunds. Our aim is to make your shopping experience as enjoyable as possible, and our dedicated customer service team is always here
